# Analyse

In [1]:
import numpy as np
from importlib import reload

## Plotter alle rådataene til magnetfeltet

In [5]:
import load_data
import plot_data
reload(load_data)
reload(plot_data)

mag_data_obj = load_data.load_magnetic_field_data()
for person, measurements in mag_data_obj.data.items():
    for name, measurement in measurements.items():

        # magnetfelt-plots bruker timestamps. Vi trenger ikke kalibreringen i starten.
        timestamps = measurement.timestamps
        start, end = load_data.get_index_stamps(measurement.data[0], (timestamps))

        plot_data.plot_one_to_many(
            x_array = measurement.data[0][start:end],
            y_arrays = [ m[start:end] for m in measurement.data[1:6]],      # 1:3 plotter bare x- og y-komponenten til B-feltet.
            legends = mag_data_obj.data_fields[1:6],                        # er ikke så nyttig å se abs(B) i plottet.
            xlabel="time [s]",
            ylabel=r"Magnetic field $[\mu T]$",
            title=f"{name}",
            filepath=f"{person}/Magnetfelt/{name}"
        )
        print(np.mean([m[start:end] for m in measurement.data[1:6]], 1))

[ -9.34036119   3.19752785 -49.40084109  50.37914865]
[ -3.17348342 -13.59449552 -48.70569801  50.66733864]
[ -2.78967159 -17.49965477 -49.3465637   52.4324632 ]
[ -9.72809414   1.41980821 -51.18475322  52.12625312]
[ -1.42276575 -19.4605292  -53.16674124  56.63546632]
[ -1.9349254  -19.48581905 -52.41969316  55.95862924]
[ -1.78682195 -19.10015578 -53.35697907  56.70279282]
[ -6.95258599  -2.99630246 -45.01025124  45.64491272]
[  0.06541049 -22.22819689 -47.41938188  52.37108826]
[  0.89495668 -23.21359001 -47.65443794  53.01570004]
[  0.62107355 -23.31389342 -47.8814582   53.25961451]


## Plotte vinkel mellom magnetfelt og referanse

In [13]:
import load_data
import plot_data
import magnetic_field_analysis as m_analysis
reload(plot_data)
reload(m_analysis)
reload(load_data)
from load_data import MEASUREMENTS


for measurement in MEASUREMENTS:
    angles_dict = m_analysis.get_magnetic_field_reference_angles_dict(measurement)
    for person, angles_array in angles_dict.items():
        plot_data.plot_one_to_one(
            x_array=angles_array[0],
            y_array=angles_array[1],
            legend=f"{person}-{measurement}-vinkel",
            xlabel="time [s]",
            ylabel=r"$\theta$ [deg]",
            title=f"{person}-{measurement}-vinkel",
            filepath=f"{person}/Vinkel/{measurement}"
        )

## Plotte inklinasjon

In [14]:
import load_data
import plot_data
import magnetic_field_analysis as m_analysis
reload(plot_data)
reload(m_analysis)
reload(load_data)


for measurement in load_data.MEASUREMENTS:
    inclination_dict = m_analysis.get_inclination_dict(measurement)
    for person, inclination_angles in inclination_dict.items():
        plot_data.plot_one_to_one(
            x_array=inclination_angles[0],
            y_array=inclination_angles[1],
            legend=f"{person}-{measurement}-vinkel",
            xlabel="time [s]",
            ylabel=r"$\theta$ [deg]",
            title=f"{person}-{measurement}-vinkel",
            filepath=f"{person}/Inclination/{measurement}"
        )

## Regne ut inklinasjon

In [3]:
import load_data
import magnetic_field_analysis as m_analysis
reload(m_analysis)
reload(load_data)


for measurement in load_data.MEASUREMENTS:
    inclination_dict = m_analysis.get_inclination_dict(measurement)
    m_analysis.analyze_results(inclination_dict, "Inclination", measurement)

### Regne ut vinkel mellom nord og Nidarosdomen/Tyholttårnet

In [16]:
import location
reload(location)

angle_dict = location.calculate_angle_north()

for key, value in angle_dict.items():
    print(key)
    for k, v in value.items():
        print(k, ": ", v)

Nidarosdomen
Oskar :  25.144180874053287
Thorb :  25.485747314333963
Vemund :  25.50664480664334
Tyholt
Oskar :  -80.03760091336322
Thorb :  -80.06043314348827
Vemund :  -80.0499053078677
Tyholt_restart
Oskar :  -80.03760091336322
Thorb :  -80.06043314348827
Vemund :  -80.0499053078677
Tyholt_flymodus
Oskar :  -80.03760091336322
Thorb :  -80.06043314348827
Vemund :  -80.0499053078677


## Regne ut posisjon

In [17]:
import location
import load_data
reload(location)
reload(load_data)

for person in load_data.PERSONS:
    location.get_location(person)

In [13]:
import load_data
import plot_data
reload(load_data)
reload(plot_data)

loc_data_obj = load_data.load_location_data()
# for person, measurement in loc_data_obj.data.items():
 
    # plot_data.plot_one_to_one(
    #     x_array = measurement.data[0],
    #     y_array = measurement.data[1],
    #     legend = loc_data_obj.data_fields[1],
    #     xlabel="time [s]",
    #     ylabel=r"Latitude",
    #     title=f"{person}",
    #     # filepath=f"{person}/{name}"
    # )
    # plot_data.plot_one_to_one(
    #     x_array = measurement.data[0],
    #     y_array = measurement.data[2],
    #     legend = loc_data_obj.data_fields[2],
    #     xlabel="time [s]",
    #     ylabel=r"Latitude",
    #     title=f"{person}",
    #     # filepath=f"{person}/{name}"
    # )

## Regne ut deklinasjon

In [4]:
import load_data
import magnetic_field_analysis as mfa
import location
reload(mfa)
reload(load_data)
reload(location)

# Regne ut vinkler (asimuter).
angle_dict = location.calculate_angle_north()

# output ligger i ./Results/Declination/
for measurement in angle_dict.keys():
    mfa.get_declination(measurement=measurement, 
                        angle_input_dict=angle_dict[measurement], 
                        num_decimals=2, 
                        save_to_file=True
                    )



### Samle alle vinklene i en array og beregne resultat

In [5]:
import magnetic_field_analysis as mfa
reload(mfa)

YALLA1 = mfa.analyze_all(ta_med_vemund_nidaros=True, num_decimals=2, save_to_file=True)
YALLA2 = mfa.analyze_all(ta_med_vemund_nidaros=False, num_decimals=2, save_to_file=True)
print(YALLA1)
print(YALLA2)

{'Declination': {'Average': -10.53, 'Median': -5.75, 'Stdev': 13.98}, 'Inclination': {'Average': 72.15, 'Median': 70.23, 'Stdev': 5.36}}
{'Declination': {'Average': -6.14, 'Median': -5.36, 'Stdev': 5.39}, 'Inclination': {'Average': 71.2, 'Median': 70.12, 'Stdev': 4.82}}


## Scatter-plot deklinasjon

In [2]:
import plot_results as ar
reload(ar)

ar.scatter_plot_declination()

<Figure size 1000x700 with 0 Axes>

## Scatter-plot Inklinasjon

In [3]:
import plot_results as ar
reload(ar)

ar.scatter_plot_inclination()

<Figure size 1000x700 with 0 Axes>